# Сбор данных для проверки пьедесталов

In [1]:
!pwd
!date
!ls

/home/sphere/SIT/Analysis/Baselines
Чт фев 13 16:53:51 MSK 2020
Baselines.ipynb  Baselines.txt.ipynb	       Parser.dat    tele.csv
Baselines.txt	 Collect_baselines_data.ipynb  Parser.ipynb  Telemetry.ipynb


In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mp
from matplotlib import dates
import datetime as dt

In [2]:
plt.rcParams['figure.figsize'] =(8,4)
plt.rcParams['lines.markersize'] = 2
plt.rcParams['lines.markeredgewidth'] = 1
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['grid.alpha'] = 0.7
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['font.size'] = 14
plt.rcParams['figure.max_open_warning'] = 129

In [3]:
''' Чтение телеметрии '''
filename = "tele.csv"
columns = ['event', 'syncro', 'tmicro', 'timesec', 'date', 'time', 'cur', 'summ']
tele = pd.read_csv(filename, header = None, index_col=None, sep='\s+', names=columns)
print(tele.shape)
# проверить наличие повторов
# True - есть повторы номеров событий
if ( True in (tele.event.value_counts().values > 1)):
    # Удалить повторы
    tele = tele.drop_duplicates(subset=['event'], keep='first')
    print("Duplicates dropped, new shape: ", tele.shape)
print("Duplicates: ", True in (tele.event.value_counts().values > 1))

tele.head()

(1065297, 8)
Duplicates:  False


,event,syncro,tmicro,timesec,date,time,cur,summ
0,57069,-1,14:06:45.674,1569074955,2019-09-21,14:09:15,106,16133.45
1,57070,-1,14:06:58.674,1569074968,2019-09-21,14:09:28,106,21243.75
2,57071,-1,14:07:12.674,1569074982,2019-09-21,14:09:42,106,20587.86
3,57072,-1,14:07:12.674,1569074982,2019-09-21,14:09:42,106,14999.95
4,57073,-1,14:08:13.674,1569075043,2019-09-21,14:10:43,103,15707.61


In [4]:
''' Чтение парсера '''
filename = "Parser.dat"
columns = ['event', 'Ianode', 'Tmos', 'Up', 'fifoerr']
info = pd.read_csv(filename, header = None, index_col=None, sep='\s+', names=columns)
print(info.shape)

# проверить наличие повторов
# True - есть повторы номеров событий
if( True in (info.event.value_counts().values > 1)):
    # Удалить повторы
    info = info.drop_duplicates(subset=['event'], keep='first')
    print("Duplicates dropped, new shape: ", info.shape)
print("Duplicates: ", True in (info.event.value_counts().values > 1))

# Удалить fifo errors
old = info.shape
info = info[info.fifoerr == 0]
print("Удалены Fifo_err\nБыло   :", old, "\nСтало  :",info.shape, "\nУдалено:  ", old[0] - info.shape[0])
# итог
info.head(2)

(1149122, 5)
Duplicates:  False
Удалены Fifo_err
Было   : (1149122, 5) 
Стало  : (1044611, 5) 
Удалено:   104511


,event,Ianode,Tmos,Up,fifoerr
0,57069,1.032,17.1,28.27,0
1,57070,1.032,17.1,28.27,0


In [5]:
''' Объединить телеметрию и парсер '''
merged = pd.merge(left=tele, right=info, left_on='event', right_on='event')
print(merged.shape)
merged.head()

(1032571, 12)


,event,syncro,tmicro,timesec,date,time,cur,summ,Ianode,Tmos,Up,fifoerr
0,57069,-1,14:06:45.674,1569074955,2019-09-21,14:09:15,106,16133.45,1.032,17.1,28.27,0
1,57070,-1,14:06:58.674,1569074968,2019-09-21,14:09:28,106,21243.75,1.032,17.1,28.27,0
2,57071,-1,14:07:12.674,1569074982,2019-09-21,14:09:42,106,20587.86,1.032,17.1,28.27,0
3,57072,-1,14:07:12.674,1569074982,2019-09-21,14:09:42,106,14999.95,1.078,17.0,28.25,0
4,57073,-1,14:08:13.674,1569075043,2019-09-21,14:10:43,103,15707.61,1.078,17.0,28.25,0


In [6]:
merged.to_csv("telemetry.scv")

In [6]:
''' Чтение пьедесталов: baselines'''
filename = "Baselines.txt"
columns = ['event']
for i in range(1, 129):
    columns.append(str(i))
base = pd.read_csv(filename, header = None, index_col=None, sep='\s+', names = columns)
print(base.shape)

# проверить наличие повторов
# True - есть повторы номеров событий
if( True in (base.event.value_counts().values > 1)):
    # Удалить повторы
    base = base.drop_duplicates(subset=['event'], keep='first')
    print("Duplicates dropped, new shape", base.shape)
print("Duplicates: ", True in (info.event.value_counts().values > 1))

# Выкинуть нули
old = base.shape
base = base.loc[(base != 0).all(axis = 1)]
print("Удалены нули\nБыло   :", old, "\nСтало  :", base.shape, "\nУдалено:  ", old[0] - base.shape[0])
# итог
print(base.shape)

base.head()

(675680, 129)
Duplicates:  False
Удалены нули
Было   : (675680, 129) 
Стало  : (582576, 129) 
Удалено:   93104
(582576, 129)


,event,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
49,57118,204.21,204.41,198.69,200.74,199.52,202.23,200.09,205.35,200.59,...,203.71,200.85,206.43,201.49,199.82,200.80,200.00,200.01,199.96,200.0
50,57119,204.16,204.48,196.90,198.89,197.78,199.62,198.06,203.71,200.41,...,203.60,201.53,203.79,199.85,199.90,200.88,200.02,200.00,199.91,200.0
51,57120,202.92,204.46,198.14,202.16,198.87,199.91,199.53,203.07,200.84,...,204.50,201.63,205.30,201.21,199.92,200.54,200.01,200.00,199.91,200.0
52,57121,204.37,204.18,198.61,200.57,200.19,200.66,201.16,204.42,202.31,...,204.62,203.20,206.39,201.62,199.87,200.74,200.00,200.04,199.92,200.0
53,57122,203.61,204.30,199.64,200.40,200.42,200.60,199.35,204.41,200.66,...,205.28,200.12,205.83,201.68,199.90,200.79,200.00,200.00,199.95,200.0


In [108]:
## Объединить инфо с пьедесталами
data = pd.merge(left=merged, right=base, left_on='event', right_on='event')
print(data.shape)
data.head(2)

(564338, 140)


,event,syncro,tmicro,timesec,date,time,cur,summ,Ianode,Tmos,...,119,120,121,122,123,124,125,126,127,128
0,57118,-1,13:02:33.672,1569153909,2019-09-22,12:05:09,104,11149.63,1.017,17.4,...,203.71,200.85,206.43,201.49,199.82,200.80,200.00,200.01,199.96,200.0
1,57120,-1,13:03:31.673,1569153910,2019-09-22,12:05:10,104,16085.05,1.017,17.4,...,204.50,201.63,205.30,201.21,199.92,200.54,200.01,200.00,199.91,200.0


In [8]:
data.to_csv("baselines.csv")